In [1]:
import os
#os.system("pip install pystan==2.19.1.1 fbprophet")
##os.system("pip3 install fbprophet --no-cache-dir --no-binary :all:")
os.system("pip uninstall slack -y")
os.system("pip uninstall slackclient -y")
os.system("pip install slack")
os.system("pip install slackclient")
os.system("pip install yfinance")
os.system("pip install yahoofinancials")
##os.system("pip install keras")
##os.system("pip install tensorflow")
#os.system("pip install statsmodels")
#os.system("pip install sklearn")

0

In [2]:
#!apt-get install build-dep python-psycopg2
#!pip install psycopg2-binary
os.system("pip install boto3")

0

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
import math
plt.style.use('fivethirtyeight')
import yfinance as yf
from yahoofinancials import YahooFinancials
import statistics
from statistics import stdev

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from numpy import log
from statsmodels.tsa.seasonal import seasonal_decompose
from fbprophet import Prophet
from datetime import date, datetime
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from subprocess import run, Popen, PIPE

from katflow.util import set_exp,get_exp,get_exp_id, search_runs
from katflow.prophet import run_model, register_model, load_model
from katflow.database import write_to_table, read_from_table, truncate_table

from slack import WebClient
from slack.errors import SlackApiError

In [ ]:
token = "your slack token"
channel = '#test'
periods_prediction = 1 #days
ticker = 'AAPL'

In [ ]:
PROJECT_SCHEMA = 'demo_yahoo_stock'
TABLE_MODEL = 'demo_yahoo_model_meta'

In [ ]:
model_df = read_from_table(PROJECT_SCHEMA,TABLE_MODEL)
model_df.head()

In [ ]:
MODEL_NAME = "yahoo_stock_pred_"+ticker+"_prophet"
if model_df[model_df['model']==MODEL_NAME].empty:
    raise ValueError("Model is not trained for " + ticker+ "ticker. Please run modelling pipeline.")
else:
    model_df = model_df[model_df['model']==MODEL_NAME].sort_values('prod_time',ascending=False).head(1)

In [ ]:
model_df

In [ ]:
exp_id = model_df['experiment_id'].values[0]
model_name = model_df['model'].values[0]
run_id = model_df['run_id'].values[0]

In [ ]:
print('experiment_id :',exp_id)
print('=='*50)
print('model name :',model_name)
print('=='*50)
print('run_id :',run_id)

In [ ]:
location = model_df.loc[0:,'location'].values[0]
location

In [ ]:
model = load_model(location)

In [ ]:
#future=model.make_future_dataframe(periods=30,freq='D',include_history=False) #as the time stamp is Quaterly & 4 is for test data records
today = date.today().strftime("%d-%m-%Y")
future = pd.date_range(start=today, periods=periods_prediction).to_frame(index = False, name='ds')
forecast=model.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].head()

In [ ]:
from matplotlib import pyplot
model.plot(forecast)
pyplot.show()

In [ ]:
forecast.to_csv("output.csv",index=False)
file = "output.csv"
title = "Amazon Revenue"
text = "Yahoo Stock Prediction"

client = WebClient(token=token)

try:
    response = client.chat_postMessage(
        channel=channel,
        text=text)
    response = client.files_upload(
        channels=channel,
        file=file,
        title=title
)
except SlackApiError as e:
    # You will get a SlackApiError if "ok" is False
    assert e.response["ok"] is False
    assert e.response["error"]  # str like 'invalid_auth', 'channel_not_found'
    print(f"Got an error: {e.response['error']}")